In [1]:
import json

In [2]:
from collections import Counter

In [188]:
import pandas as pd

In [145]:
with open('main.json', encoding='utf-8') as fh:
    data = json.load(fh)



In [196]:
dframe = pd.DataFrame(columns=["table","condition","field","operator","value"])

In [197]:
dframe

,table,condition,field,operator,value


In [123]:
for each_key in list(data.keys()):
    sample = data[each_key]
    
    print("\n\n",each_key)
    while 1>0:
        if len(sample['rules'])>0:
            if 'rules' in list(sample.keys()):
                #print(sample['rules'])
                
                for each in sample['rules'][:-1]:
                    print(str(sample['condition']))
                    if "value" in list(each.keys()):
                        print("yeah")
                        dframe.append({"table":str(each_key),"condition":str(sample['condition']),"field":str(each['field']),"operator":str(each['operator']),"value":str(each['value'])},ignore_index=True)

                

                sample = list(sample['rules'])[-1]
        else:
            break
            
            
print(dframe)



 account
and
and
yeah
or
or
or
or


 customer
and
and
yeah
or
or
or
or
Empty DataFrame
Columns: [table, condition, field, operator, value]
Index: []


In [167]:
cols =["table","condition","field","operator","value"]

In [193]:
x = data['account']
dic = {}

for each in cols:

    dic[each] = 5

new_row = pd.Series(data=dic)

dframe.append(dic,ignore_index=True)
print(dframe)

Empty DataFrame
Columns: [table, condition, field, operator, value]
Index: []


In [204]:
row = pd.Series([1,2,3,4,5])
dframe.append(row,ignore_index=True)

,table,condition,field,operator,value,0,1,2,3,4
0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,3.0,4.0,5.0
